In [12]:
import numpy as np
import pandas as pd
import re
import calendar

import seaborn as sns
import matplotlib.style as style


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

style.use('fivethirtyeight')

In [15]:
df = pd.read_csv('outputs/job_postings_data_scientist_all_cities.csv')
df.head()

,job_title,salary,company,location,is_remote,job_rating,job_summary,post_date,extract_date,job_url
0,Data Scientist,"$102,663 - $157,709 a year",National Security Agency,"Fort Meade, MD",NaN,4.2,Relevant experience must be in designing/imple...,25 days ago,2021-01-10,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,NaN,Orangetheory Fitness,"Washington, DC",NaN,3.4,"Experience in machine learning, analytics and/...",2 days ago,2021-01-10,https://www.indeed.com/rc/clk?jk=60ca99bf7f8b5...
2,Data Scientist,"$76,228 - $162,994 a year",Valsatech Corp,"Washington, DC",Temporarily remote,4.6,Data manipulation and data engineering experie...,3 days ago,2021-01-10,https://www.indeed.com/company/Valsatech-Corp/...
3,Data Scientist,"$120,000 - $160,000 a year",Urbane Systems LLC,"Washington, DC",Temporarily remote,NaN,Investigate and implement new scientific analy...,25 days ago,2021-01-10,https://www.indeed.com/company/Urbane-Systems-...
4,Federal - Data Scientist,NaN,Accenture,"Washington, DC",NaN,4.0,"Experience with analytics, statistical modelin...",3 days ago,2021-01-10,https://www.indeed.com/rc/clk?jk=5c1bfa7eaea6e...


In [16]:
def getting_to_know(df, question=None):
    display(df.shape)
    display(df.columns)
    if question == 'y':
        display(df.head())
        display(df.tail())
    print('--------------------------END--------------------------------')
getting_to_know(df)

(721, 10)

Index(['job_title', 'salary', 'company', 'location', 'is_remote', 'job_rating',
       'job_summary', 'post_date', 'extract_date', 'job_url'],
      dtype='object')

--------------------------END--------------------------------


In [17]:
# Missing values
def missing_values_func(df):
    """
    @author: Cristobal Zamorano Astudillo
    
    Personalize Missing Data Function
    
    Paramaters
    ----------
    df : DataFrame of interest
    
    Returns
    -------
    A string with counting all the features of the input DataFrame. If some values are missing, then 
    function will return a DataFrame with the following:
    - Index as the features with the missing values
    - A `Missing Values Count` Feature that tells the exact number of rows that has a misssing value in that feature index.
    - A `% of Total Values` that tells how much are the missing values of that feature with respect to the other missing values.
    """
    
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values Count', 1 : '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)


    # Returns
    if mis_val_table_ren_columns.empty:
        return f'Your selected dataframe has  {df.shape[1]} features. There are  {mis_val_table_ren_columns.shape[0]} features that have missing values.'
    else:
        display(mis_val_table_ren_columns.style.background_gradient(cmap='Reds'))
        return f'Your selected dataframe has  {df.shape[1]} features. There are  {mis_val_table_ren_columns.shape[0]} features that have missing values.'
        
missing_values_func(df)

,Missing Values Count,% of Total Values
salary,635,88.1
is_remote,615,85.3
job_rating,206,28.6


'Your selected dataframe has  10 features. There are  3 features that have missing values.'

In [18]:
df['job_title'].value_counts()

Data Scientist                                     128
Senior Data Scientist                               32
Senior Data Analyst                                 13
Data Scientist, Mid                                  9
Senior Statistician                                  8
                                                  ... 
SENIOR DATA ANALYST - PATIENT-CENTERED RESEARCH      1
Network Engineer/Data Scientist                      1
Senior Data Analyst - Health                         1
Data Analyst I - Data Science Team                   1
Research Scientist - Machine Learning                1
Name: job_title, Length: 395, dtype: int64

In [19]:
df['salary'].value_counts()

$102,663 - $157,709 a year    8
$126,000 - $165,000 a year    7
$72,351 - $90,157 a year      6
$70,000 a year                4
$122,530 - $159,286 a year    3
$150,000 - $190,000 a year    2
$60 - $65 an hour             2
$60 - $80 an hour             2
$73,973 - $112,240 a year     2
$125,000 - $150,000 a year    2
$117,265 - $165,084 a year    2
$175,000 - $190,000 a year    2
$79,521 - $90,000 a year      2
$90,000 - $130,000 a year     2
$90,000 - $150,000 a year     2
$100,000 - $200,000 a year    2
$75,000 - $140,000 a year     2
$102,663 - $142,958 a year    2
$87,198 - $134,798 a year     2
$145,000 - $190,000 a year    1
$120,000 - $160,000 a year    1
$92,211 - $169,987 a year     1
$155,000 - $160,000 a year    1
$51,102 - $81,263 a year      1
$76,228 - $162,994 a year     1
$105,000 - $150,000 a year    1
From $75,000 a year           1
From $55,000 a year           1
$150,000 - $200,000 a year    1
$96,516 - $135,395 a year     1
$121,316 - $157,709 a year    1
$70,000 

In [22]:
df['is_remote'].value_counts()  

Remote                69
Temporarily remote    37
Name: is_remote, dtype: int64

In [21]:
df.iloc[402]

job_title                                     Senior Statistician
salary                                                        NaN
company                                     DevTech Systems, Inc.
location                                           Washington, DC
is_remote                                                     NaN
job_rating                                                    3.4
job_summary     The Senior Statistician will develop technical...
post_date                                             26 days ago
extract_date                                           2021-01-10
job_url         https://www.indeed.com/rc/clk?jk=294d2a3a548dd...
Name: 402, dtype: object